In [1]:
import numpy as np
from jetbot import Robot
from jetbot import Camera
import cv2
print(cv2.__version__)
import asyncio
from PIL import Image
from IPython.display import clear_output
import matplotlib.pyplot as plt
import ipywidgets.widgets as widgets
import traitlets
from jetbot import bgr8_to_jpeg

4.1.1


In [2]:
robot = Robot()
#camera = Camera.instance(width=1280, height=720)
#camera = Camera.instance(width=1024,height=800)
#camera = Camera.instance()
#net = cv2.dnn.readNet('data/one_sphere/yolov3.weights','data/one_sphere/yolov3.cfg' )

In [3]:
from datetime import datetime

In [4]:
from PIL import Image

In [6]:
# print(camera)

In [5]:
from jetbot import Robot
import ipywidgets.widgets as widgets

robot = Robot()
import numpy as np
import cv2
CLASSES = ["sphere"]

COLORS = np.random.uniform(0,255,size=(len(CLASSES),3))

net = cv2.dnn.readNetFromTensorflow('data/fisheye_graphs/frozen_inference_graph.pb', 'data/fisheye_graphs/final_graph.pbtxt')


def gstreamer_pipeline(
    capture_width=1280,
    capture_height=720,
    display_width=1280,
    display_height=720,
    framerate=30,
    flip_method=0,
):
    return (
        "nvarguscamerasrc ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

import cv2
import matplotlib.pyplot as plt
image = 0
cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
print(gstreamer_pipeline(flip_method=0))

nvarguscamerasrc ! video/x-raw(memory:NVMM), width=(int)1280, height=(int)720, format=(string)NV12, framerate=(fraction)30/1 ! nvvidconv flip-method=0 ! video/x-raw, width=(int)1280, height=(int)720, format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink


In [9]:

image_trait = widgets.Image(format='png', width=300, height=300)

i=0
while cap.isOpened() and i <30:
    boolCaptured, gt = cap.read()
    input_image = cv2.cvtColor(gt, cv2.COLOR_RGBA2RGB)
    h,w,c = input_image.shape
    print("Image input size: " + str(h) + ", " + str(w) + ", c: " + str(c))
    blob = cv2.dnn.blobFromImage(image=cv2.resize(input_image,(w,h)), scalefactor=1.0, size=(300,300), swapRB=True)
    net.setInput(blob)
    detections = net.forward()
    for j in np.arange(0, detections.shape[2]):
        confidence = detections[0,0,j,2]
        if confidence > 0.70:
            print(confidence)
            idx = int(detections[0,0,j,1])
            box = detections[0,0,j,3:7] * np.array([w,h,w,h])
            (startX, startY, endX, endY) = box.astype("int")
            print("Id: " + str(idx))
            label = "{}: {:.2f}%".format(CLASSES[0], confidence*100)
            print(label)
            cv2.rectangle(input_image, (startX,startY), (endX,endY), COLORS[0],2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(input_image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[0],2)
            print("Detected " + str(CLASSES[0]) +  " at X position: " + str(startX))
            #starting x of the detected object on the right side
            print("startX " + str(startX))
            print("w" + str(w/2))
            if startX > w/2:
                print("startX bigger than half image size")
                #apply velocity to the left jetbot's joint
                robot.right(speed=0.2)
                robot.left(speed=0)
                robot.stop()
            elif startX < w/2:
                #apply velocity to the right jetbot's joint
                robot.left(speed=0.2)
                robot.right(speed=0)
                robot.stop()
#         else:
#             robot.left(speed=0.1)
#             robot.right(speed=0) 
    robot.stop()
    _, encoded_image = cv2.imencode('.png', input_image)
    bytes_img_obj = encoded_image.tobytes()
    gt_widget = widgets.Image(value=bytes_img_obj, format="png",width=300, height=300)
    camera_link = traitlets.dlink((gt_widget,'value'), (image_trait, 'value'))
    clear_output(wait=True)
    display(image_trait)
    i+=1

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\x00\x00\x00\x02\xd0\x08\x02\x00\x00\x00@\x1fJ\x0…

In [ ]:
robot.stop()
cap.release()

Main changes applied so far: changing the layer batch optimazation FusedBatchNormV3 to FusedBatchNorm because of incongruences in tensorflow module version